This notebook is intended to show the efforts of Team ErrorPlane importing contrail images into the Hopper enviroment and converting the files into .jpegs

In [ ]:
import json
import os

This next cell is a Google written cell. It parses a landsat_filename and returns the full path to the img

In [ ]:
# starting the file definitions this is the 1st cell (100% Google written cell)
def path_from_filename(landsat_filename):
  """Returns the full GCP filepath for the provided `landsat_filename`.

  An example filename looks like:
  LC08_L1TP_221074_20181218_20181227_01_T1_B10.TIF

  The returned path ends up looking like:
  gs://gcp-public-data-landsat/LC08/01/221/074/LC08_L1TP_221074_20181218_20181227_01_T1/LC08_L1TP_221074_20181218_20181227_01_T1_B10.TIF

  Args:
    landsat_filename: str, file name of a landsat scene.
  Returns:
    str, full path to the corresponding landsat scene.
  """
  split_name = landsat_filename.split('_')
  path, row = split_name[2][:3], split_name[2][3:]
  bands = split_name[0]  # Either LC08 or LT08
  parent_dir = '_'.join(split_name[:-1])
  return os.path.join(f'gs://gcp-public-data-landsat/{bands}/01/',
                      path, row, parent_dir, landsat_filename)

gsutil is a Python application that lets you access Cloud Storage from the command line. gsutil performs all operations, including uploads and downloads, using HTTPS and transport-layer security (TLS).

This library is what Google leveraged in their own notebooks. Refinement of this process would identify different methods in pulling in the data when not operating in a jupyter notebook

In [ ]:
# Trial cell to check is gsutil is working and there is access allowed in the desired location.
# !gsutil cp gs://landsat_contrails_dataset/2021_06_11_1623455786/data/landsat_contrails.json-00051-of-00100 tmp/landsat_contrails.json-00051-of-00100

The Google json files total 100. Each team member took a small chunck to import into the system, hence the lack of DRY code. The next code block was run multiple times. Each time is it was run, the json file should be incremented. There are a handful of files that failed to import. Futher work can be exploring why these failed.

In [ ]:

#needs to be re run to update json data file: json-00001-00100 - json-00100-00100



# !gsutil cp gs://landsat_contrails_dataset/2021_06_11_1623455786/data/landsat_contrails.json-00051-of-00100 /tmp/landsat_contrails.json-00051-of-00100
# cd /projects/raggarw/ErrorPlane/

f = open('/tmp/landsat_contrails.json-00051-of-00100','r') # opens 1st json file in read mode from GCP

tif_dir_path1 = r'data/tif_image_files/images_without_contrails'
tif_dir_path2 = r'data/tif_image_files/images_with_contrails'
tif_dir_path3 = r'data/tif_image_files/annotated_images_with_contrails'

# m.p written cell do not need to run this cell more than 1 time 
tiff1 = open(tif_dir_path1,'a') # this file is going to hold all the tif image names without contrails
tiff2 = open(tif_dir_path2,'a') # this file is going to hold all the tif image names with contrails
tiff3 = open(tif_dir_path3,'a')# this files is going to hold all the tif images with contrails and annotated


"""
m.p. added code/ cell. This cell uses the json files from the GCP  
1.Reads in one line at a time (each line is 1 json object) 
2.Removes advected flight information
3.Test for the presence of polygons, splits files accordingly
4.Saves folders to Google drive
"""
tifnames_with_contrail_list = []
tifnames_without_contrail_list = []

# (FOR TEAM USING CODE DO NOT NEED TO RUN THIS CELL)
# (THIS CELL IS FOR PROCESSING JSON FILES AND LOADING FILES INTO THEIR RESPECTIVE DIRECTORIES )
counter = 0
number_of_json_files = 36
  
indiv_json_file = json.loads(f.readline()) # reads in 1st line as a python object
while counter <= number_of_json_files:
   # while indiv_json_file != "": (this statement does not work so used a counter)
   counter = counter +1
   indiv_json_file.pop('advected_flight_waypoints') # deletes flight waypoint
   indiv_json_file.pop('advected_flight_density') # deletes flight density
   #print(indiv_json_file) # verifying deleted objects
   #print(type(indiv_json_file))# json should be a dictionary, verifying file types
   #print(type(indiv_json_file['polygons']))
   image_filename = indiv_json_file['filename'] # retrieve the image filename 
   landsat_image_name = indiv_json_file['filename'] # retrieve the image filename again for filters
   
   if len(indiv_json_file['polygons'])== 0: # check for polygon data and separate
     tiff1.write(landsat_image_name) # write a file listing of tif file names
     tiff1.write(os.linesep)
     tifnames_without_contrail_list.append(landsat_image_name)
     dir_name = 'data/json_image_files/filenames_without_contrails'
     fileType = '.json'
     image_filename_list = image_filename.split(".")
     image_filename = image_filename_list[0]+ fileType
     path = os.path.join(dir_name, image_filename)
     #print(path) verifying path 
     cf = open(path,'w')
     indiv_json_file = json.dumps(indiv_json_file, indent=2) # m.p. added cell this re-creates json file of the parsed files (file name & polygons only)
     cf.write(indiv_json_file)
   else:
     tiff2.write(landsat_image_name) # write a file listing of tif file names
     tiff2.write(os.linesep)
     tifnames_with_contrail_list.append(landsat_image_name)
     dir_name = 'data/json_image_files/filenames_with_contrails'
     fileType = '.json'
     image_filename_list = image_filename.split(".")
     image_filename = image_filename_list[0]+ fileType
     path = os.path.join(dir_name, image_filename)
     #print(path) verifying path
     ncf = open(path, 'w') # open a file to hold filenames that have contrails
     indiv_json_file = json.dumps(indiv_json_file, indent= 2)
     ncf.write(indiv_json_file)
      
   #print(image_filename) # verify file names
   indiv_json_file = json.loads(f.readline())

f.close()

cf.close()
ncf.close()
tiff1.close()
tiff2.close()
tiff3.close()



# at this point should have all of the tiff filenames in lists and the json files in sep directories

Run the next two cells to verify number of files in the share repository so far. There is no true total count that our team is aware of.

In [ ]:
current_picture_list = [] # creating list to hold TIFF file names
list_of_polygons = [] # creating list to hold polygon data
file_count = 0 

# this code walks through the directory and opens each file
directory1 = '/projects/raggarw/ErrorPlane/json_image_files/filenames_with_contrails'
for filename in os.listdir(directory1):
  f= open(os.path.join(directory1,filename))
  indiv_json_file = json.loads(f.read()) # reads file in as a python object
  current_picture = indiv_json_file['filename'] # extract TIF file name
  current_picture_list.append(current_picture) # adds file name to list
  list_of_polygons.append(indiv_json_file['polygons']) # extract and append polygon data
  file_count = file_count + 1




In [ ]:
print(file_count)
print(current_picture_list[0])
print(list_of_polygons[0])
# THIS CELL IS FOR VERIFICATION FILE DIRECTORY IS READ

### Downloading and converting .tif to .jpeg

The files we imported are just json files, next we need to identify the correct scenes from the Google labled contrail dataset. Once we have the right scene, download the correct .tif file and then convert it to .jpeg. 

In [ ]:
file_contrailYES = open("data/json_image_files/images_with_contrails", "r")
file_contrailNO = open("data/json_image_files/images_without_contrails", "r")

# reading the file
dataYES = file_contrailYES.read()
contrail_yes = dataYES.split()
file_contrailYES.close()
# printing the data

dataNO = file_contrailNO.read()
contrail_no = dataNO.split()
file_contrailNO.close()

In [ ]:
print("These are images that have a contrail")
contrail_yes[:5]

In [ ]:
print("These are images do not have a contrail")
contrail_no[:5]

Now that we have a list of YES_contrails and NO_contrails, lets checki the functionality of the Google provided function.

In [ ]:
path_from_filename("LC08_L1TP_019036_20181125_20181210_01_T1_B10.TIF")

Awesome, it provides the url to the img. Now lets download all of them

In [ ]:
# Downloading the .TIF files

#NO CONTRAILS
for name in contrail_no:
    !gsutil cp {path_from_filename(name)} data/tif_image_files/without_contrails/

In [ ]:
#YES CONTRAILS
for name in contrail_yes:
    !gsutil cp {path_from_filename(name)} data/tif_image_files/with_contrails/

Lastly we have to convert the .tif file into a .jpeg. We'll leverage the libraries PIL and glob in our efforts to do this. Lets first check we can remove the file tag at the end of the name.

In [ ]:
"data/tif_image_files/with_contrails/LC08_L1TP_006026_20180826_20180830_01_T1_B10.TIF"[:-4]

The orginal code was written on a Windows machine and they see forward slashes in path names a bit differently. We needed to replace all "\\" with "/". We need to gather all the paths to YES_contrails and NO_contrails.

In [ ]:
YES_path = glob.glob('data/tif_image_files/with_contrails/*.*')
YES_path = [i.replace("\\","/") for i in YES_path]

In [ ]:
NO_path = glob.glob('data/tif_image_files/without_contrails/*.*')
NO_path = [i.replace("\\","/") for i in NO_path]

Now we need to convert the .TIF files into .jpeg. We'll use PIL for that. Snippet borrowed from this Stackoverflow.  

https://stackoverflow.com/questions/43978819/convert-tiff-i16-to-jpg-with-pil-pillow

In [ ]:
for file in YES_path:
    image = Image.open(file)
    image.mode = 'I'
    image.point(lambda i:i*(1./256)).convert('L').save(file[:-4] + '.jpeg')
#     print(file)

In [ ]:
for file in NO_path:
    image = Image.open(file)
    image.mode = 'I'
    image.point(lambda i:i*(1./256)).convert('L').save(file[:-4] + '.jpeg')
#     print(file)

And now our preprocessing is done. We have converted to .jpegs and divided our binary classification groups into their respective folders.